**10. Generar un RDD en el que cada tupla tenga el formato (key, value) donde:**

**a) key sea una palabra del léxico de la wikipedia**

**b) value sea una lista donde cada elemento de la misma sea una tupla de dos
elementos:**

  **i. identificador de contenido donde aparezca esa palabra.**

  **ii. la frecuencia con la que aparece esa palabra en ese contenido.**


In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-le

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Bajo el csv **contents_text_sample**

In [ ]:
downloaded = drive.CreateFile({'id':"1XDhQ8I1rm2MIXXOD4fzAEMNhAD1Eya4m"})
downloaded.GetContentFile('contents_text_sample.csv') 

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Leo el CSV:

In [ ]:
!pip install pyarrow
texts = pd.read_csv('contents_text_sample.csv')
texts.to_parquet('contents_text_sample.parquet')
del texts

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.parquet('contents_text_sample.parquet')
rdd = df.rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
rdd.take(5)

[Row(id=50, title='Anacardiaceae', text="{{Ficha de taxón\n| name = Anacardiáceas\n| image = Gui1 cashewfruit2.jpg\n| image_caption = ''[[Anacardium occidentale]]''\n| regnum = [[Plantae]]\n| divisio  = [[Magnoliophyta]]\n| classis  = [[Magnoliopsida]]\n| ordo = [[Sapindales]]\n| familia = '''Anacardiaceae'''\n| familia_authority = [[Robert Brown|R.Br.]]\n| subdivision_ranks = Géneros\n| subdivision = \nVer texto\n}}\n'''Anacardiaceae''' es una [[familia (biología)|familia]] de [[Plantae|plantas]] esencialmente [[árbol|arbóreas]] y [[arbusto|arbustivas]] perteneciente al [[Orden (biología)|orden]] [[Sapindales]]. La constituyen 77 [[género (biología)|géneros]] con unas 700 especies aceptadas, de las casi 3000 descritas, propias de países tropicales, cálidos y templados.<ref>[http://www.theplantlist.org/1.1/browse/A/Anacardiaceae/ Anacardiaceae en The Plant List, vers. 1.1, 2013]</ref>\n\n== Descripción ==\nSon árboles, arbustos, raramente subarbustos o [[trepadora]]s, frecuentemente co

Las palabras las vamos a obtener de "text", por lo tanto las que son *None* no nos interesan.

Mapeo para quedarme solo con el id y el texto lo spliteo:

In [ ]:
datos = rdd.filter(lambda x: x.text != None).map(lambda x: (x.id, x.text.split()))

Ahora hago un  flap-map, donde cada clave será la palabra y el valor será el id del contenido donde aparece y su frecuencia en el mismo

In [ ]:
d = datos.flatMap(lambda x: [(palabra,(x[0],x[1].count(palabra))) for palabra in set(x[1])])
d.take(1)

[('[[hermafrodita]]s.', (50, 1))]

Reduzco por clave:
(hay una aclaración luego de la ejecución)

In [ ]:
d.reduceByKey(lambda a,b: a+b).take(1)

[("''k''",
  (160,
   1,
   4218,
   2,
   9030,
   1,
   21223,
   1,
   26964,
   1,
   38839,
   1,
   47192,
   2,
   48817,
   2,
   60352,
   2,
   102906,
   21,
   104853,
   1,
   116180,
   1,
   176528,
   1,
   216399,
   1,
   368594,
   1,
   829382,
   1,
   971490,
   1,
   1098046,
   2,
   1280318,
   1,
   1309393,
   2,
   1648107,
   1,
   2253153,
   1,
   2469677,
   1,
   2823812,
   1,
   4239239,
   1,
   4534157,
   1,
   5880884,
   1,
   6063600,
   1,
   6209707,
   1,
   6540170,
   1,
   6619481,
   4,
   7284626,
   1,
   7376433,
   2,
   7512971,
   1,
   7659560,
   1,
   8695286,
   1,
   8727330,
   1,
   8786981,
   1,
   9027462,
   1,
   9512014,
   2,
   9774071,
   2))]

# **ACLARACIÓN:** 
No es exactamente lo pedido, sino que se están anidando todos los numeros (id,frecuencia,id2,frecuencia2,....) y no de a tuplas.

No pude lograr que sea como lo pedido, lo había pensado de esta forma pero se rompe:

In [ ]:
a = (1,2)
b = (3,4)
(a,b)

((1, 2), (3, 4))

In [ ]:
rdd = d.reduceByKey(lambda a,b: (a,b))

rdd.take(1)

Py4JJavaError: ignored